In [1]:
def clear_memory():
    !sudo sysctl -w vm.drop_caches=3
clear_memory()

Coloque su dedo en el lector de huellas dactilares
vm.drop_caches = 3


In [2]:
import pandas as pd
import polars as pl
import ManipulateData
import MetodoLexico
import DataLoader_Polars
from rapidfuzz import fuzz
import jellyfish
import re
import matplotlib.pyplot as plt

In [3]:
data_loader = DataLoader_Polars.DataLoaderPolars("../DataSets_CSVs/DataSets_Parquets/")
df_concesiones = data_loader.load_concesiones()
#manipulator = ManipulateData.ManipulateData(df_concesiones, "", "")

In [4]:
titulares_unicos = df_concesiones.sort('titular').select('titular').unique()

In [5]:
def date_string_to_date(df):
    month_map = {
        "enero":1, "febrero":2, "marzo":3, "abril":4, "mayo":5, "junio":6,
        "julio":7, "agosto":8, "septiembre":9, "octubre":10, "noviembre":11, "diciembre":12
    }
    df = (
        df.with_columns(
            ur_norm = pl.col("ultimo_registro")
                    .str.strip_chars()
                    .str.replace_all(r"\s+", " ")
                    .str.to_lowercase()
        ).with_columns(
            # extrae tokens "mes" y "año"
            mes_txt = pl.col("ur_norm").str.split(" ").list.get(0),
            anio    = pl.col("ur_norm").str.split(" ").list.get(-1).cast(pl.Int16)
        ).with_columns(
            mes = pl.col("mes_txt").replace(month_map)
        ).with_columns(
            ultimo_registro = pl.date(pl.col("anio"), pl.col("mes"), pl.lit(1))
        ).drop(["ur_norm","mes_txt"])
    )
    return df

In [6]:
df_concesiones = date_string_to_date(df_concesiones)

In [7]:
# Para truncar fechas que ya tienen tiempo
df_concesiones = df_concesiones.with_columns(
    pl.col('ultimo_registro').dt.date().alias('ultimo_registro')
)

In [8]:
#n_nombres_titulares_antes_estandarizar = len(df_concesiones.select('titular').unique())
#n_nombres_titulares_antes_estandarizar

In [9]:
'''
df_concesiones = ManipulateData.ManipulateData(df_concesiones).estandarizar_titular('titular',
                                                                                    sin_espacios=False,
                                                                                    sin_stopword=True,
                                                                                    sin_terminos_mercantiles=True,
                                                                                    quitar_tokens_1_letra=True)
'''

"\ndf_concesiones = ManipulateData.ManipulateData(df_concesiones).estandarizar_titular('titular',\n                                                                                    sin_espacios=False,\n                                                                                    sin_stopword=True,\n                                                                                    sin_terminos_mercantiles=True,\n                                                                                    quitar_tokens_1_letra=True)\n"

In [10]:
#n_nombres_titulares_despues_estandarizar = len(df_concesiones.select('titular').unique())
#n_nombres_titulares_despues_estandarizar

In [11]:
# Seleccionar columnas en el orden deseado

df_concesiones = df_concesiones.select([
    'titulo', 'titular', 'ultimo_registro',
    *[col for col in df_concesiones.columns if col not in ['titulo', 'titular', 'ultimo_registro']]
])


In [12]:
# Ordenar el DataFrame
df_concesiones = df_concesiones.sort([
    pl.col('titulo').count().over('titulo'),
    'titular', 
    'ultimo_registro'
], descending=[True, False, False])

In [13]:
# Obtener información de títulos y titulares
titulos_info = df_concesiones.group_by('titulo').agg([
    pl.col('titular').n_unique().alias('N_titulares')
]).sort('N_titulares', descending=True)



In [14]:
volumenes_totales_info = df_concesiones.group_by('titulo').agg([
    pl.col('volumen_de_extraccion_anual_de_aguas_nacionales_que_ampara_el_titulo_en_m3').n_unique().alias('cambios_volumen'),
    pl.col('volumen_de_extraccion_anual_de_aprovechamientos_subterraneos_en_m3').n_unique().alias('cambios_volumen_subterraneo'),
    pl.col('volumen_de_extraccion_anual_de_aprovechamientos_superficiales_en_m3').n_unique().alias('cambios_volumen_superficial')
])

In [15]:
periodos_info = df_concesiones.group_by('titulo').agg([
    pl.col('ultimo_registro').n_unique().alias('cantidad_periodos'),
    pl.col('ultimo_registro').unique().alias('periodos_presentes'),
    pl.col('ultimo_registro').count().alias('N_cambios_ultimo_registro'),
])

In [16]:

# Recrear df_anomalias con los datos
df_anomalias = pl.DataFrame({
    'titulo': titulos_info['titulo'].to_list(),
    'N_titulares': titulos_info['N_titulares'].to_list(),
    'Score': [None] * len(titulos_info),
    'Diferente_titular': [None] * len(titulos_info),
    'Caracteres_distintos': [None]* len(titulos_info),
    'Cambios_en_20_periodos': periodos_info['N_cambios_ultimo_registro'].to_list(), # Cuantas veces aparece el registro en los 20 periodos
    'Periodos': periodos_info['periodos_presentes'].to_list(), # aqui se ponen los periodos en los que hubieron cambios.
    'Cantidad_periodos': periodos_info['cantidad_periodos'].to_list(), #Cuantos periodos hay en total en dicho registro
    'Anomalia_V_Total': volumenes_totales_info['cambios_volumen'].to_list(),
    'Anomalia_V_Subterraneo': volumenes_totales_info['cambios_volumen_subterraneo'].to_list(),
    'Anomalia_V_Superficial': volumenes_totales_info['cambios_volumen_superficial'].to_list()
})

In [17]:
'''
def length_prefilter(a: str, b: str, base_abs: int = 5, rel_long: float = 0.50) -> bool:
    # Semántica: devuelve True si las longitudes son tan distintas
    # que podemos DESCARTAR el par sin seguir a métodos léxicos.
    L1 = len(a or "")
    L2 = len(b or "")
    Lmax = max(L1, L2)
    if Lmax <= 3:
        return False  # no decidas por longitud cuando son ultra cortos

    # Umbral absoluto dinámico (20% de Lmax, al menos base_abs)
    abs_thr = max(base_abs, int(round(0.20 * Lmax)))

    # Umbral relativo: más exigente en cortos
    rel_thr = 0.40 if Lmax < 10 else rel_long  # p.ej. 0.40 para cortos, 0.50 para largos

    gap_abs = abs(L1 - L2)
    gap_rel = gap_abs / Lmax

    return (gap_abs > abs_thr) and (gap_rel > rel_thr)
'''

'\ndef length_prefilter(a: str, b: str, base_abs: int = 5, rel_long: float = 0.50) -> bool:\n    # Semántica: devuelve True si las longitudes son tan distintas\n    # que podemos DESCARTAR el par sin seguir a métodos léxicos.\n    L1 = len(a or "")\n    L2 = len(b or "")\n    Lmax = max(L1, L2)\n    if Lmax <= 3:\n        return False  # no decidas por longitud cuando son ultra cortos\n\n    # Umbral absoluto dinámico (20% de Lmax, al menos base_abs)\n    abs_thr = max(base_abs, int(round(0.20 * Lmax)))\n\n    # Umbral relativo: más exigente en cortos\n    rel_thr = 0.40 if Lmax < 10 else rel_long  # p.ej. 0.40 para cortos, 0.50 para largos\n\n    gap_abs = abs(L1 - L2)\n    gap_rel = gap_abs / Lmax\n\n    return (gap_abs > abs_thr) and (gap_rel > rel_thr)\n'

In [18]:
'''
def score_y_clase(a:str, b:str):
    # Pre-filtrado por longitud
    if length_prefilter(a, b):
        return 0, 0
    else:
        #metricas
        ts = fuzz.token_set_ratio(a, b)
        jw = jellyfish.jaro_winkler_similarity(a, b) * 100
        score = (ts + jw) / 2
        #reglas    
        if ts >= 96 or (ts >= 93 and jw >= 93):
            clase = 1 #iguales
        elif ts < 90 or (ts < 85 and jw < 88):
            clase = 0 #diferentes
        else:
            clase = 2 
    return score, clase
'''

'\ndef score_y_clase(a:str, b:str):\n    # Pre-filtrado por longitud\n    if length_prefilter(a, b):\n        return 0, 0\n    else:\n        #metricas\n        ts = fuzz.token_set_ratio(a, b)\n        jw = jellyfish.jaro_winkler_similarity(a, b) * 100\n        score = (ts + jw) / 2\n        #reglas    \n        if ts >= 96 or (ts >= 93 and jw >= 93):\n            clase = 1 #iguales\n        elif ts < 90 or (ts < 85 and jw < 88):\n            clase = 0 #diferentes\n        else:\n            clase = 2 \n    return score, clase\n'

In [19]:
def score_y_clase_wrapper(a: str, b: str):
    """Wrapper para usar MetodoLexico en lugar de la función score_y_clase"""
    metodo = MetodoLexico.MetodoLexico(a, b)
    return metodo.score_y_clase()


In [20]:
def get_all_metrics(a: str, b: str):
    """Función que retorna todos los valores de una vez para evitar múltiples instanciaciones"""
    metodo = MetodoLexico.MetodoLexico(a, b)
    score, clase, caracteres_conflictivos = metodo.score_y_clase()
    return (score, clase, caracteres_conflictivos)


In [21]:
# Títulos con exactamente 2 titulares únicos
tit2 = (
    df_concesiones
    .select(['titulo','titular'])
    .unique()
    .group_by('titulo')
    .agg(pl.col('titular').unique().alias('titulares'))
    .with_columns(pl.col('titulares').list.len().alias('N_titulares'))
    .filter(pl.col('N_titulares') == 2)
)

# Extrae pares (t1, t2)
tit2_pairs = tit2.with_columns([
    pl.col('titulares').list.get(0).alias('titular1'),
    pl.col('titulares').list.get(1).alias('titular2')
]).select(['titulo','titular1','titular2'])

In [22]:
tit2_scored = (
    tit2_pairs.with_columns([
        pl.struct(['titular1','titular2']).map_elements(
            lambda s: get_all_metrics(s['titular1'], s['titular2']),
            return_dtype=pl.Object  # <- importante si all_metrics es tuple/list
        ).alias('all_metrics')
    ])
    .with_columns([
        pl.col('all_metrics').map_elements(lambda x: x[0]).alias('Score'),
        pl.col('all_metrics').map_elements(lambda x: x[1]).alias('Diferente_titular'),
        pl.col('all_metrics').map_elements(lambda x: x[2]).alias('Caracteres_distintos'),
    ])
    .drop('all_metrics')
)


/tmp/ipykernel_121295/3898457111.py:8: MapWithoutReturnDtypeWarning: 'return_dtype' of function python_udf must be set

A later expression might fail because the output type is not known. Set return_dtype=pl.self_dtype() if the type is unchanged, or set the proper output data type.
  .with_columns([


In [23]:
df_anomalias = (
    df_anomalias.drop(['Score','Diferente_titular', 'Caracteres_distintos'])  # Eliminar las columnas anteriores
    .join(tit2_scored.select(['titulo','Score','Diferente_titular','Caracteres_distintos']), on='titulo', how='left')  # Incluir Caracteres_distintos
    .select([
        'titulo',
        'N_titulares',
        'Score',
        'Diferente_titular',
        'Caracteres_distintos',
        'Cambios_en_20_periodos',
        'Periodos',
        'Cantidad_periodos',
        'Anomalia_V_Total',
        'Anomalia_V_Subterraneo',
        'Anomalia_V_Superficial'
    ])
)

In [24]:

# fechas = df_concesiones.select('ultimo_registro').unique().sort('ultimo_registro')
# titulos_list = df_concesiones.group_by('titulo').agg(pl.count()).sort('count', descending=True)
# nombres_unicos_en_titulares = df_concesiones.sort('titular').select('titular').unique()
# Distribución de títulos por frecuencia
#dist_titulos_list = titulos_list.group_by('count').agg(pl.count().alias('frequency')).sort('count',descending=True)

In [25]:
'''
plt.bar(dist_titulos_list['count'].to_list(), dist_titulos_list['frequency'].to_list())
plt.yscale('log')
plt.show()
'''

"\nplt.bar(dist_titulos_list['count'].to_list(), dist_titulos_list['frequency'].to_list())\nplt.yscale('log')\nplt.show()\n"

In [27]:
clear_memory()

Coloque su dedo en el lector de huellas dactilares
La huella no coincide
Coloque su dedo en el lector de huellas dactilares
vm.drop_caches = 3


In [28]:
df_subterraneas = data_loader.load_subterraneas()

In [29]:
df_superficiales = data_loader.load_superficiales()